# Drug interaction prediction in *E. coli*
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Compound class stratified validation](#classlockout)
* [Choose thresholds for interactions](#threshclass)
* [Precision-recall and ROC curves](#pr)
* [Probability calibration](#calib)
* [Predictions on the test set](#test)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import label_binarize

sys.path.append('..')
import base.chemgen_utils as utl
import MLmod.predictor as prd

<a id="intro"></a> 
## Introduction
It has been previously shown that drug interactions can be predicted in bacteria using chemogenomic data. Here we use random forest classifier on single-compound chemical genetics data in *E. coli* to predict antagonisms, synergies and additive combinations. 

We encode our input predictor matrix as follows: 
+ For each combination load single-compound profiles. Each profile has dimensions `(1 x genes)` and the following possible gene states {-1,0,+1}. Negative drug-gene interaction (-1) implies increased sensitivity in that gene deletion, while positive gene state (+1) indicates decreased sensitivity in that mutant
+ Combination profiles are generaeted based on superposition of individual drug profiles and may take on the following gene states {-2, -1, 0, +1, +2, +/-}
+ We then use one-hot encoding scheme ("dummy variable encoding") before passing the predictor matrix `X` to the classifier

We furthermore subset the data so that only those genes are included that are significantly enriched in antagonisms and synergies based on chi-squared test. This gene list is `interaction-genes-Ecoli`

In [ ]:
drugleg_fname = "../data/chemicals/legend_gramnegpos.txt"
gene_subset = '../data/interaction-genes-Ecoli'
gene_subset = pd.read_csv(gene_subset, header=None)[0].values

In [ ]:
X_chemgen = pd.read_csv('../data/chemgenetics/nichols_signed.csv', index_col=0)
X_chemgen = X_chemgen.iloc[:,np.where(np.isin(X_chemgen.columns, gene_subset))[0]]
targets = pd.read_csv("../data/chemgenetics/nichols_y.csv")
combs = targets['comb'].values
y = targets['type'].values

X_df = pd.DataFrame([utl.get_comb_feat_signed(X_chemgen, c) for c in combs])

Since we are using `OneVsRestClassifier`, we convert our categorical response variable `y` ("synergy" ,"antagonism", "none") into a `n x 3` binary array using `label_binarize` function

In [ ]:
# one vs rest classification
y[y=='none'] = 0
y[y=='antagonism']=1
y[y=='synergy']=2

y=y.astype('int')
y = label_binarize(y, classes=[0, 1, 2])

Encode combination profiles using one-hot encoding scheme:

In [ ]:
X_onehot = pd.get_dummies(X_df.astype('category'))

In [ ]:
X_onehot.iloc[:6,:12]

In [ ]:
# at least 5 combinations with that variable set
X_onehot = X_onehot.loc[:,(X_onehot.sum(axis=0) > 4)]

We performed grid search to find the best parameters for the `RandomForestClassifier`:

In [ ]:
param_dict = {'n_estimators': 300,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_depth': None,
 'class_weight': None}

<a id="classlockout"></a> 
## Compound Class Stratified Validation
In order to assess the generalization error we stratified the data by compound class:

In [ ]:
drugclasses = pd.read_csv(drugleg_fname, sep='\t')
druglegend = drugclasses.loc[:,['Drug', 'Class']]

comb_drugs = pd.DataFrame(np.array([utl.split_vec(i) for i in combs]),
                          columns=['d1', 'd2'])
comb_drugs = utl.add_class(strain=comb_drugs,
                           druglegend=druglegend)
# an array with all drug class labels
class_arr = np.unique(np.union1d(pd.unique(comb_drugs.class1),
                                 pd.unique(comb_drugs.class2)))

In [ ]:
comb_drugs[:10]

In each cross-validation iteration we withhold the data of a certain drug class and test the trained model on the withheld combinations:

In [ ]:
pr = prd.MultiClassPredictions(X=X_onehot.to_numpy(), y=y,
                                   combs=combs,
                                  **param_dict,
                                   clf='randomforest',
                                   top = 30)

In [ ]:
pr.crossval_drugclass(class_arr=class_arr, leg_class=comb_drugs)

In [ ]:
'''pr.save_topfeat(outdir='../data/chemgenetics/', 
                fname="topfeat-multiclass-Nichols-signed",
                    featname=X_onehot.columns.values)'''

In [ ]:
auc_df = (pd.concat({k: pd.DataFrame(v.values(),
                                   index=['AUCROC_none',
                                          'AUCROC_antag',
                                          'AUCROC_syn']).T \
                   for k,v in pr.auc.items()}).
         reset_index().rename(columns={"level_0": "cvfold"}).
         drop(columns=["level_1"]))

ap_df = (pd.concat({k: pd.DataFrame(v.values(),
                           index=['AP_none',
                                  'AP_antag',
                                  'AP_syn']).T \
           for k,v in pr.avprec.items()}).
 reset_index().rename(columns={"level_0": "cvfold"}).
 drop(columns=["level_1"]))

metrics = pd.merge(auc_df, ap_df, on='cvfold', how='inner')

This table shows the cross-validation results sorted by average precision for antagonism prediction:

In [ ]:
(metrics.
 sort_values('AP_antag', ascending=False).
 reset_index(drop=True))

We can do the same for synergy prediction - arrange the table by average precision:

In [ ]:
(metrics.
 sort_values('AP_syn', ascending=False).
 reset_index(drop=True))

We can check the most important genetic features and rank them by the number of cross-validation folds in which these appeared as top 30 features based on the splitting criterion (Gini impurity):

In [ ]:
topvars = (pd.concat(pr.topfeat).
                   reset_index().
                   rename(columns={"level_0": "cvfold"}).
                   drop(columns=['level_1']))

In [ ]:
featname=X_onehot.columns.values

In [ ]:
topvars = (topvars.assign(feature=featname[topvars.feat]).
           drop(columns=['feat']))

**Top genes for antagonism prediction**

In [ ]:
(topvars[topvars.type == 'antagonism'].
 groupby('feature').agg('count').
 query('cvfold > 1').
 sort_values('cvfold', ascending=False).iloc[:30,0])

**Top genes for synergy prediction**

In [ ]:
(topvars[topvars.type == 'synergy'].
 groupby('feature').agg('count').
 query('cvfold > 1').
 sort_values('cvfold', ascending=False).iloc[:30,0])

<a id="threshclass"></a> 
## Choose Thresholds for Antagonisms and Synergies
The 'probability' score $p_{RF}$ output by random forests does not correspond to the probability of being a synergy or antagonism, i.e. 
$$p_{RF}(c=C|X) \neq P(c=C|X)$$

By construction random forests do not approximate class probabilities (unlike logistic regression e.g.) and due to class imbalance (80% additive combinations, 10% synergies, 10% antagonisms) almost all combinations are predicted to be neutral if one takes 
$$ \hat{C} = \mathrm{argmax} (p_{RF}(c|X)) $$

Since we are using `OneVsRestClassifier` however, we have technically 3 different binary classifiers, one for each combination type. We can use precision-recall characteristics in cross-validation folds to find thresholds for $p_{RF}(c=\mathrm{antagonism}|X)$ and $p_{RF}(c=\mathrm{synergy}|X)$.

In each cross-validation fold select thresholds such that precision > 0.6:

In [ ]:
# choose prob score cutoff such that precision > 0.6
prec_thresh = 0.6

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score

TP_df = list()
for cl in pr.predicted.keys():
    # print(cl)
    ycl = y[np.isin(combs, pr.predicted[cl]['comb'].values)]
    gt = pd.DataFrame(ycl, columns=['none', 'antagonism', 'synergy'])
    gt['comb'] = combs[np.isin(combs, pr.predicted[cl]['comb'].values)]
    pred_df = pd.merge(left=pr.predicted[cl], right=gt, how='inner', on='comb')
    
    precision, recall, thresh = precision_recall_curve(pred_df['antagonism'].values,
                                                        pred_df['prob_ant'].values)
    if np.any(precision > prec_thresh):
        # maximum recall
        rmax = np.max(recall[precision > prec_thresh])
        # maximum precision
        pmax = np.max(precision[recall == rmax])
        # index with maximum recall and precision > prec_thresh
        idx = np.where(np.logical_and(precision == pmax, recall == rmax))[0][0]
        # corresponding threshold
        antag_thresh = thresh[idx] if idx < len(thresh) else thresh[-1]
        antag_tp = pred_df[(pred_df.prob_ant > antag_thresh) == pred_df.antagonism]
        antag_tp = antag_tp[antag_tp.antagonism == 1]
        antag_tp['thresh'] = antag_thresh
        antag_tp['precision'] = pmax
        antag_tp['recall'] = rmax
        antag_tp['cvfold'] = cl
        TP_df.append(antag_tp)
    
    precision, recall, thresh = precision_recall_curve(pred_df['synergy'].values,
                                                        pred_df['prob_syn'].values)
    if np.any(precision > prec_thresh):
        # maximum recall
        rmax = np.max(recall[precision > prec_thresh])
        # maximum precision
        pmax = np.max(precision[recall == rmax])
        # index with maximum recall and precision > prec_thresh
        idx = np.where(np.logical_and(precision == pmax, recall == rmax))[0][0]
        # corresponding threshold
        syn_thresh = thresh[idx] if idx < len(thresh) else thresh[-1]

        syn_tp = pred_df[(pred_df.prob_syn > syn_thresh) == pred_df.synergy]
        syn_tp = syn_tp[syn_tp.synergy == 1]
        syn_tp['thresh'] = syn_thresh
        syn_tp['precision'] = pmax
        syn_tp['recall'] = rmax
        syn_tp['cvfold'] = cl
        TP_df.append(syn_tp)

In [ ]:
TP_df = pd.concat(TP_df).reset_index(drop=True)

Thresholds to call antagonisms in cross-validation folds:

In [ ]:
np.unique(TP_df[TP_df['antagonism']==1]['thresh'].values)

As we can see in most cases we end up with precision greater than 0.6 if we take a cutoff between 0.25-0.4. This already suggests  that $p_{RF}$ cannot be interpreted as probability $P(C|X)$ as we will see again in [Probability calibration](#calib) section. We can take the median as a cutoff for calling antagonisms:

In [ ]:
antag_thresh = np.median(np.unique(TP_df[TP_df['antagonism']==1]['thresh'].values))
antag_thresh

Thresholds to call synergies in cross-validation:

In [ ]:
np.unique(TP_df[TP_df['synergy']==1]['thresh'].values)

Again we achieve precision greater than 0.6 if we take relatively loose cutoffs between 0.16-0.35. We'll take the median as our threshold for calling synergies:

In [ ]:
syn_thresh = np.median(np.unique(TP_df[TP_df['synergy']==1]['thresh'].values))
syn_thresh

In [ ]:
#TP_df.to_csv('Nichols-true-positives-CVfold.csv')

<a id="pr"></a> 
## Plot Precision-Recall and ROC curves

In [ ]:
pr.auc.keys()

In [ ]:
drug_classes = ['DNA_gyrase', 'RNA_polymerase', 'aminoglycoside',
                'beta-lactam', 'chloramphenicol', 'folic_acid_biosynthesis',
               'human_drug', 'macrolide', 'multiple', 'other_DNA',
               'other_cell_wall', 'oxidative_stress', 'tRNA',
               'tetracycline']

In [ ]:
import matplotlib
font = {'family': 'normal',
        'weight': 'normal',
        'size': 18}
matplotlib.rc('font', **font)
fig, ax = plt.subplots(figsize=(6,6))
for cl in drug_classes:
    plt.plot(pr.recall[cl][1], pr.precision[cl][1], lw=2, alpha=0.75,
         label='%s (AP = %0.2f)' % (cl, pr.avprec[cl][1]))
    plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
ax.yaxis.get_major_ticks()[1].label1.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#plt.title(title)
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={"size": 13})
plt.legend([], frameon=False)
plt.tight_layout()
#plt.savefig('Ecoli-antagonism-vs-rest.pdf')

In [ ]:
figsize = (6, 6)
fig_leg = plt.figure(figsize=figsize)
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), loc='center', frameon=False)
# hide the axes frame and the x/y labels
ax_leg.axis('off')
#fig_leg.savefig('Ecoli-legend-antag.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
for cl in drug_classes:
    plt.plot(pr.recall[cl][2], pr.precision[cl][2], lw=2, alpha=0.75,
         label='%s (AP = %0.2f)' % (cl, pr.avprec[cl][2]))
    plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
ax.yaxis.get_major_ticks()[1].label1.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.legend([], frameon=False)
plt.tight_layout()
#plt.savefig('Ecoli-synergy-vs-rest.pdf')

In [ ]:
figsize = (6, 6)
fig_leg = plt.figure(figsize=figsize)
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), loc='center', frameon=False)
# hide the axes frame and the x/y labels
ax_leg.axis('off')
#fig_leg.savefig('Ecoli-legend-synergy.pdf')

Add only mean average precision:

In [ ]:
preds_ = (pd.concat(pr.predicted).
                  reset_index().
                  rename(columns={"level_0": "cvfold"}).
                  drop(columns=['level_1']))

In [ ]:
y_gt = targets.loc[:, ['comb', 'type']]
y_gt['syn'] = 0
y_gt['ant'] = 0
y_gt.loc[y_gt.type == 1,'ant'] = 1
y_gt.loc[y_gt.type == 2,'syn'] = 1

In [ ]:
pred_vs_true = pd.merge(preds_, y_gt, on='comb')

In [ ]:
pred_vs_true = pred_vs_true[np.isin(pred_vs_true.cvfold, drug_classes)]

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

# A "micro-average": quantifying score on all classes jointly
prec_micro, recall_micro, _ = precision_recall_curve(pred_vs_true['ant'].values,
                                                    pred_vs_true['prob_ant'].values)
ap_micro = average_precision_score(pred_vs_true['ant'].values,
                                   pred_vs_true['prob_ant'].values,
                                   average="micro")

In [ ]:
from matplotlib.lines import Line2D
fig, ax = plt.subplots(figsize=(6.5,6.5))
for cl in drug_classes:
    plt.plot(pr.recall[cl][1], pr.precision[cl][1],
             color='grey', lw=1.5, alpha=0.6)
    plt.xlim([-0.05, 1.05])
plt.plot(recall_micro, prec_micro,
        label='Aggregated (AP = {0:0.2f})'
               ''.format(ap_micro),
         color='#f54248', linestyle='-', linewidth=3)

handles, labels = ax.get_legend_handles_labels()
patch = Line2D([0], [0], color='grey', linewidth=2, linestyle="-",
               label='Drug class withheld')
handles.append(patch) 
plt.ylim([-0.01, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
#ax.yaxis.get_major_ticks()[0].label1.set_visible(False)
ax.yaxis.get_major_ticks()[1].label1.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#plt.title(title)
plt.legend(handles=handles, loc='upper right', prop={"size": 13},
          frameon=False)
plt.tight_layout()
#plt.savefig('Ecoli-antagonism-vs-rest-grey.pdf')

In [ ]:
# A "micro-average": quantifying score on all classes jointly
prec_micro, recall_micro, _ = precision_recall_curve(pred_vs_true['syn'].values,
                                                    pred_vs_true['prob_syn'].values)
ap_micro = average_precision_score(pred_vs_true['syn'].values,
                                   pred_vs_true['prob_syn'].values,
                                   average="micro")

In [ ]:
fig, ax = plt.subplots(figsize=(6.5,6.5))
for cl in drug_classes:
    plt.plot(pr.recall[cl][2], pr.precision[cl][2],
            color='grey', lw=1.5, alpha=0.6)
    plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.plot(recall_micro, prec_micro,
        label='Aggregated (AP = {0:0.2f})'
               ''.format(ap_micro),
         color='#f54248', linestyle='-', linewidth=3)

handles, labels = ax.get_legend_handles_labels()
patch = Line2D([0], [0], color='grey', linewidth=2, linestyle="-",
               label='Drug class withheld')
handles.append(patch) 
plt.ylim([-0.02, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
ax.yaxis.get_major_ticks()[1].label1.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#plt.title(title)
plt.legend(handles=handles, loc='upper right',
           prop={"size": 13},
            #bbox_to_anchor=(1.2,0.8),
          frameon=False)
#plt.legend([], frameon=False)
plt.tight_layout()
#plt.savefig('Ecoli-synergy-vs-rest-grey.pdf')

Plot ROC curves:

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
for cl in drug_classes:
    plt.plot(pr.fpr[cl][1], pr.tpr[cl][1], lw=2, alpha=0.7,
             label='{0} (area = {1:0.2f})'
             ''.format(cl, pr.auc[cl][1]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
ax.yaxis.get_major_ticks()[1].label1.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.legend([], frameon=False)
#plt.savefig('Ecoli-ROC-antagonism-vs-rest.pdf')

In [ ]:
figsize = (6, 6)
fig_leg = plt.figure(figsize=figsize)
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), loc='center', frameon=False)
# hide the axes frame and the x/y labels
ax_leg.axis('off')
#fig_leg.savefig('Ecoli-ROC-legend-antag.pdf')

ROC curves for synergy vs rest:

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
for cl in drug_classes:
    plt.plot(pr.fpr[cl][2], pr.tpr[cl][2], lw=2, alpha=0.7,
             label='{0} (area = {1:0.2f})'
             ''.format(cl, pr.auc[cl][2]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
ax.yaxis.get_major_ticks()[1].label1.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.legend([], frameon=False)
#plt.savefig('Ecoli-ROC-synergy-vs-rest.pdf')

In [ ]:
figsize = (6, 6)
fig_leg = plt.figure(figsize=figsize)
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), loc='center', frameon=False)
# hide the axes frame and the x/y labels
ax_leg.axis('off')
#fig_leg.savefig('Ecoli-ROC-legend-syn.pdf')

In [ ]:
fpr_micro, tpr_micro, _ = roc_curve(pred_vs_true['ant'].values,
                                                    pred_vs_true['prob_ant'].values)
roc_auc_micro = auc(fpr_micro, tpr_micro)

In [ ]:
fig, ax = plt.subplots(figsize=(6.5,6.5))
for cl in drug_classes:
    plt.plot(pr.fpr[cl][1], pr.tpr[cl][1],
             color='grey', lw=1.5, alpha=0.6)
plt.plot(fpr_micro, tpr_micro,
        label='Aggregated (AUCROC = {0:0.2f})'
               ''.format(roc_auc_micro),
         color='#f54248', linestyle='-', linewidth=3)

handles, labels = ax.get_legend_handles_labels()
patch = Line2D([0], [0], color='grey', linewidth=2, linestyle="-",
               label='Drug class withheld')
handles.append(patch) 
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
ax.yaxis.get_major_ticks()[0].label1.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.legend(handles=handles, loc='lower right', prop={"size": 13},
          frameon=False)
plt.tight_layout()
#plt.savefig('Ecoli-ROC-antagonism-vs-rest-grey.pdf')

In [ ]:
fpr_micro, tpr_micro, _ = roc_curve(pred_vs_true['syn'].values,
                                                    pred_vs_true['prob_syn'].values)
roc_auc_micro = auc(fpr_micro, tpr_micro)

In [ ]:
fig, ax = plt.subplots(figsize=(6.5,6.5))
for cl in drug_classes:
    plt.plot(pr.fpr[cl][2], pr.tpr[cl][2],
             color='grey', lw=1.5, alpha=0.6)
plt.plot(fpr_micro, tpr_micro,
        label='Aggregated (AUCROC = {0:0.2f})'
               ''.format(roc_auc_micro),
         color='#f54248', linestyle='-', linewidth=3)

handles, labels = ax.get_legend_handles_labels()
patch = Line2D([0], [0], color='grey', linewidth=2, linestyle="-",
               label='Drug class withheld')
handles.append(patch) 
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
#ax.xaxis.get_major_ticks()[0].label1.set_visible(False)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.get_major_ticks()[0].label1.set_visible(False)
plt.legend(handles=handles, loc='lower right', prop={"size": 13},
          frameon=False)
plt.tight_layout()
#plt.savefig('Ecoli-ROC-synergy-vs-rest-grey.pdf')

Plot average precision (AP) vs interaction rate for each cross-validation fold:

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
# for antagonism predictions
for cl in drug_classes:
    ycl = y[np.isin(combs, pr.predicted[cl]['comb'].values)].argmax(axis=1)
    antag_rate = np.sum(ycl == 1) / ycl.size
    plt.scatter(antag_rate, pr.avprec[cl][1], marker='o', 
             label=cl, s=50)
    plt.xlim((0,0.8))
    plt.ylim((0,0.8))
plt.plot([0.01, 1], [0.01, 1], 'k--', lw=1)
plt.xlabel('Proportion of antagonisms')
plt.ylabel('Average precision')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.get_major_ticks()[0].label1.set_visible(False)
plt.tight_layout()
#plt.savefig('Ecoli-AP-antag-vs-rate.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
# for antagonism predictions
for cl in drug_classes:
    ycl = y[np.isin(combs, pr.predicted[cl]['comb'].values)].argmax(axis=1)
    syn_rate = np.sum(ycl == 2) / ycl.size
    plt.scatter(syn_rate, pr.avprec[cl][2], marker='o', 
             label=cl, s=50)
    plt.xlim((0,0.8))
    plt.ylim((0,0.8))
plt.plot([0.01, 1], [0.01, 1], 'k--', lw=1)
plt.xlabel('Proportion of synergies')
plt.ylabel('Average precision')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.get_major_ticks()[0].label1.set_visible(False)
plt.tight_layout()
#plt.savefig('Ecoli-AP-syn-vs-rate.pdf')

<a id="calib"></a> 
## Probability calibration

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
clf = OneVsRestClassifier(RandomForestClassifier(bootstrap=True,
                                                max_features='sqrt',
                                                **param_dict,
                                                random_state=2305,
                                              n_jobs=-1))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_onehot, y, test_size=0.2,
                                                    random_state=2305)

In [ ]:
clf = clf.fit(X_train, y_train)

In [ ]:
probas_ = clf.predict_proba(X_val)

Plot precision-recall for the training set:

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
precision = dict()
recall = dict()
thresh = dict()
average_precision = dict()

In [ ]:
n_classes = 3
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_val[:, i], probas_[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    precision[i], recall[i], thresh[i] = precision_recall_curve(y_val[:, i],
                                                        probas_[:, i])
    average_precision[i] = average_precision_score(y_val[:, i], probas_[:, i])


In [ ]:
from itertools import cycle
class_names = ['none', 'antagonism', 'synergy']
colors = cycle(['#808080','#FFCC33', '#009999'])


plt.figure(figsize=(10,10))
f_scores = np.linspace(0.2, 0.8, num=4)

for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y_ = f_score * x / (2 * x - f_score)
    plt.plot(x[y_ >= 0], y_[y_ >= 0], color='gray', alpha=0.2,
             label='iso-F1 curves')
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y_[45] + 0.02))
for i, color in zip(range(n_classes), colors):
    plt.plot(recall[i], precision[i], color=color, lw=2,
             label='Precision-recall of class {0} (area = {1:0.2f})'
             ''.format(class_names[i], average_precision[i]))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc="lower right")

In [ ]:
from sklearn.calibration import calibration_curve
fraction_of_positives = dict()
mean_predicted_value = dict()
for i in range(n_classes):
    proba_val = clf.predict_proba(X_val)[:, i]
    fraction_of_positives[i], mean_predicted_value[i] = calibration_curve(y_val[:,i],
                                                                proba_val,
                                                                n_bins=6)

In [ ]:
mean_predicted_value[2]

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 6))
plt.plot(mean_predicted_value[0], fraction_of_positives[0], 's-', label='none')
plt.plot([0, 1], [0, 1], '--', color='gray')
plt.plot(mean_predicted_value[1], fraction_of_positives[1], 's-', label='antagonism')
plt.plot(mean_predicted_value[2], fraction_of_positives[2], 's-', label='synergy')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.title('Uncalibrated probabilities')
plt.legend()

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
fraction_of_positives = dict()
mean_predicted_value = dict()
for i in range(n_classes):
    #proba_val = clf.predict_proba(X_val)[:, i]
    clf_calib = CalibratedClassifierCV(clf.estimators_[i], cv=5, method='sigmoid')
    proba_val = clf_calib.fit(X_train, y_train[:,i]).predict_proba(X_val)[:,1]
    fraction_of_positives[i], mean_predicted_value[i] = calibration_curve(y_val[:,i],
                                                                proba_val,
                                                                n_bins=5)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 6))
plt.plot(mean_predicted_value[0], fraction_of_positives[0], 's-', label='none')
plt.plot([0, 1], [0, 1], '--', color='gray')
plt.plot(mean_predicted_value[1], fraction_of_positives[1], 's-', label='antagonism')
plt.plot(mean_predicted_value[2], fraction_of_positives[2], 's-', label='synergy')
plt.title('Calibrated probabilities')
plt.legend()

Somehow calibrated probabilities are worse than the "uncalibrated" ones.

<a id="test"></a> 
## Generate Predictions on the Test Set

In [ ]:
X_drugs = pd.read_csv('../data/chemgenetics/nichols_testset_signed.csv', index_col=0)

In [ ]:
X_drugs = X_drugs.iloc[:,np.where(np.isin(X_drugs.columns, gene_subset))[0]]

In [ ]:
X_drugs.shape

In [ ]:
test_drugs = X_drugs.index.values

In [ ]:
import itertools
combs_test = list(itertools.combinations(test_drugs, 2))
combs_test = np.array([i[0]+"_"+i[1] for i in combs_test])

In [ ]:
len(combs_test)

In [ ]:
X_test = pd.DataFrame([utl.get_comb_feat_signed(X_drugs, c) for c in combs_test])

In [ ]:
X_test = pd.get_dummies(X_test.astype('category'))

In [ ]:
X_test.iloc[:10,:12]

Pad some columns (as some are all zeros in the test set):

In [ ]:
X_test.shape

In [ ]:
cols_pad = np.setdiff1d(X_onehot.columns, X_test.columns)

In [ ]:
for col in cols_pad:
    X_test[col] = 0

In [ ]:
X_test = X_test.loc[:,np.isin(X_test.columns, X_onehot.columns)]

In [ ]:
X_test = X_test[X_onehot.columns]

In [ ]:
np.all(X_test.columns == X_onehot.columns)

In [ ]:
# without probability calibration
y_test_proba = clf.fit(X_onehot, y).predict_proba(X_test)
antag = combs_test[y_test_proba[:,1] > antag_thresh]
syn = combs_test[y_test_proba[:,2] > syn_thresh]

In [ ]:
np.setdiff1d(antag, syn)

In [ ]:
np.setdiff1d(syn, antag)

In [ ]:
np.intersect1d(antag, syn)

In [ ]:
prob_uncalibr = pd.DataFrame(y_test_proba, index=combs_test,
             columns=['none', 'antag', 'synergy'])

In [ ]:
prob_uncalibr.to_csv('nichols_test_pred.csv')

In [ ]:
prob_uncalibr.sort_values('antag', ascending=False).iloc[:20,:]

In [ ]:
prob_uncalibr.sort_values('synergy', ascending=False).iloc[:20,:]

In [ ]:
'''from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=6)

# with calibration
probs = dict()
for i in range(n_classes):
    clf_calib = CalibratedClassifierCV(clf.estimators_[i], cv=skf, method='isotonic')
    probs[i] = clf_calib.fit(X_onehot, y[:,i]).predict_proba(X_test)'''